In [38]:
%%writefile julia_set.cu
/*
Interesting Coordinates:

(0.3, 0.2) - Spiraling tendrils
(-0.4, 0.6) - Intricate branches
(0.285, 0.01) - Deep self-similar patterns (current default)
(-0.8, 0.156) - Feathery structures
0.11230583918508258 -0.17749724081559552
*/
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <stdbool.h>
#include <cuda_runtime.h>
#include <math.h>

#define WIDTH 1920
#define HEIGHT 1080
#define MAX_ITER 256
#define PI 3.14159265358979323846

// Color palette for beautiful visualization
struct Color {
    unsigned char r, g, b;
};

// Generate smooth color gradient with more vibrant colors
__host__ __device__ Color getColor(int iter, int max_iter) {
    if (iter == max_iter) {
        return {0, 0, 0};  // Black for points in the set
    }

    // Smooth coloring with vibrant palette using sine waves
    float t = (float)iter / max_iter;

    Color c;
    // Vibrant psychedelic colors
    c.r = (unsigned char)(255 * (0.5 + 0.5 * sin(6.28318 * t + 0.0)));
    c.g = (unsigned char)(255 * (0.5 + 0.5 * sin(6.28318 * t + 2.094)));
    c.b = (unsigned char)(255 * (0.5 + 0.5 * sin(6.28318 * t + 4.189)));

    return c;
}

// CPU implementation - sequential Julia set computation
void juliaSetCPU(Color *image, int width, int height,
                 double centerX, double centerY, double zoom,
                 double cReal, double cImag, int maxIter) {
    double scale = 4.0 / (zoom * width);

    for (int py = 0; py < height; py++) {
        for (int px = 0; px < width; px++) {
            // Map pixel to complex plane (this is our starting z)
            double zReal = centerX + (px - width / 2.0) * scale;
            double zImag = centerY + (py - height / 2.0) * scale;

            int iter = 0;

            // Iterate: z = z^2 + c (c is constant, z varies by pixel)
            while (zReal*zReal + zImag*zImag <= 4.0 && iter < maxIter) {
                double zRealTemp = zReal*zReal - zImag*zImag + cReal;
                zImag = 2*zReal*zImag + cImag;
                zReal = zRealTemp;
                iter++;
            }

            image[py * width + px] = getColor(iter, maxIter);
        }
    }
}

// GPU kernel - parallel Julia set computation
__global__ void juliaSetGPU(Color *image, int width, int height,
                             double centerX, double centerY, double zoom,
                             double cReal, double cImag, int maxIter) {
    int px = blockIdx.x * blockDim.x + threadIdx.x;
    int py = blockIdx.y * blockDim.y + threadIdx.y;

    if (px >= width || py >= height) return;

    double scale = 4.0 / (zoom * width);

    // Map pixel to complex plane (this is our starting z)
    double zReal = centerX + (px - width / 2.0) * scale;
    double zImag = centerY + (py - height / 2.0) * scale;

    int iter = 0;

    // Iterate: z = z^2 + c (c is constant, z varies by pixel)
    while (zReal*zReal + zImag*zImag <= 4.0 && iter < maxIter) {
        double zRealTemp = zReal*zReal - zImag*zImag + cReal;
        zImag = 2*zReal*zImag + cImag;
        zReal = zRealTemp;
        iter++;
    }

    image[py * width + px] = getColor(iter, maxIter);
}

// Save image as PPM file
void savePPM(const char *filename, Color *image, int width, int height) {
    FILE *fp = fopen(filename, "wb");
    if (fp == NULL) {
        printf("Error: Could not open file %s\n", filename);
        return;
    }
    fprintf(fp, "P6\n%d %d\n255\n", width, height);
    fwrite(image, sizeof(Color), width * height, fp);
    fclose(fp);
    printf("Saved: %s\n", filename);
}

int main(int argc, char *argv[]) {
    // Default parameters
    int cpuStartFrame = 1;
    int cpuEndFrame = 15;
    int gpuStartFrame = 1;
    int gpuEndFrame = 15;
    int cycleLength = 100;
    double centerX = 0.285;
    double centerY = 0.01;
    double zoomStart = 1.0;
    double zoomEnd = 500.0;
    double animationStopPercent = 0.7;  // Stop c-parameter animation at 70% zoom
    bool renderCPU = true;
    bool renderGPU = true;

    // Parse command line arguments
    for (int i = 1; i < argc; i++) {
        if (strcmp(argv[i], "--cpu-start") == 0 && i + 1 < argc) {
            cpuStartFrame = atoi(argv[++i]);
        } else if (strcmp(argv[i], "--cpu-end") == 0 && i + 1 < argc) {
            cpuEndFrame = atoi(argv[++i]);
        } else if (strcmp(argv[i], "--gpu-start") == 0 && i + 1 < argc) {
            gpuStartFrame = atoi(argv[++i]);
        } else if (strcmp(argv[i], "--gpu-end") == 0 && i + 1 < argc) {
            gpuEndFrame = atoi(argv[++i]);
        } else if (strcmp(argv[i], "--start") == 0 && i + 1 < argc) {
            cpuStartFrame = gpuStartFrame = atoi(argv[++i]);
        } else if (strcmp(argv[i], "--end") == 0 && i + 1 < argc) {
            cpuEndFrame = gpuEndFrame = atoi(argv[++i]);
        } else if (strcmp(argv[i], "--cpu-only") == 0) {
            renderGPU = false;
        } else if (strcmp(argv[i], "--gpu-only") == 0) {
            renderCPU = false;
        } else if (strcmp(argv[i], "--cycle-length") == 0 && i + 1 < argc) {
            cycleLength = atoi(argv[++i]);
        } else if (strcmp(argv[i], "--zoom") == 0 && i + 1 < argc) {
            zoomEnd = atof(argv[++i]);
        } else if (strcmp(argv[i], "--centerX") == 0 && i + 1 < argc) {
            centerX = atof(argv[++i]);
        } else if (strcmp(argv[i], "--centerY") == 0 && i + 1 < argc) {
            centerY = atof(argv[++i]);
        } else if (strcmp(argv[i], "--animation-stop") == 0 && i + 1 < argc) {
            animationStopPercent = atof(argv[++i]);
            if (animationStopPercent < 0.0) animationStopPercent = 0.0;
            if (animationStopPercent > 1.0) animationStopPercent = 1.0;
        } else if (strcmp(argv[i], "--help") == 0 || strcmp(argv[i], "-h") == 0) {
            printf("Usage: %s [OPTIONS]\n\n", argv[0]);
            printf("Frame Range Options:\n");
            printf("  --start N           Set both CPU and GPU start frame (default: 1)\n");
            printf("  --end N             Set both CPU and GPU end frame (default: 15)\n");
            printf("  --cpu-start N       CPU start frame (default: 1)\n");
            printf("  --cpu-end N         CPU end frame (default: 15)\n");
            printf("  --gpu-start N       GPU start frame (default: 1)\n");
            printf("  --gpu-end N         GPU end frame (default: 15)\n");
            printf("  --cpu-only          Skip GPU rendering\n");
            printf("  --gpu-only          Skip CPU rendering\n\n");
            printf("Animation Options:\n");
            printf("  --cycle-length N    Animation cycle length in frames (default: 100)\n");
            printf("  --zoom N            Maximum zoom level (default: 500.0)\n");
            printf("  --centerX N         Zoom center X coordinate (default: 0.285)\n");
            printf("  --centerY N         Zoom center Y coordinate (default: 0.01)\n");
            printf("  --animation-stop N  Stop c-parameter animation at N percent (0.0-1.0, default: 0.7)\n");
            printf("                      Use 0.0 to animate throughout, 1.0 for no animation\n\n");
            printf("Other:\n");
            printf("  --help, -h          Show this help message\n\n");
            printf("Examples:\n");
            printf("  %s --start 1 --end 100\n", argv[0]);
            printf("  %s --cpu-start 1 --cpu-end 10 --gpu-start 1 --gpu-end 100\n", argv[0]);
            printf("  %s --gpu-only --start 1 --end 200\n", argv[0]);
            printf("  %s --cpu-end 5 --gpu-end 50 --zoom 1000\n", argv[0]);
            printf("  %s --animation-stop 0.3  (stop animation at 30%% zoom)\n", argv[0]);
            printf("  %s --animation-stop 1.0  (no animation at all)\n\n", argv[0]);
            printf("Interesting zoom locations:\n");
            printf("  --centerX 0.3 --centerY 0.2      (Spiraling tendrils)\n");
            printf("  --centerX -0.4 --centerY 0.6     (Intricate branches)\n");
            printf("  --centerX 0.285 --centerY 0.01   (Deep self-similar patterns)\n");
            printf("  --centerX -0.8 --centerY 0.156   (Feathery structures)\n");
            return 0;
        } else {
            printf("Unknown option: %s\n", argv[i]);
            printf("Use --help for usage information\n");
            return 1;
        }
    }

    // Validate parameters
    if (cpuStartFrame < 1 || cpuEndFrame < cpuStartFrame) {
        printf("Error: Invalid CPU frame range (start: %d, end: %d)\n", cpuStartFrame, cpuEndFrame);
        return 1;
    }

    if (gpuStartFrame < 1 || gpuEndFrame < gpuStartFrame) {
        printf("Error: Invalid GPU frame range (start: %d, end: %d)\n", gpuStartFrame, gpuEndFrame);
        return 1;
    }

    if (cycleLength < 1) {
        printf("Error: Cycle length must be >= 1\n");
        return 1;
    }

    if (zoomEnd <= zoomStart) {
        printf("Error: Zoom end (%.1f) must be greater than zoom start (%.1f)\n", zoomEnd, zoomStart);
        return 1;
    }

    int cpuNumFrames = cpuEndFrame - cpuStartFrame + 1;
    int gpuNumFrames = gpuEndFrame - gpuStartFrame + 1;

    printf("===========================================\n");
    printf("JULIA SET: CPU vs GPU Performance\n");
    printf("Resolution: %dx%d (%d megapixels)\n", WIDTH, HEIGHT, (WIDTH*HEIGHT)/1000000);
    printf("===========================================\n\n");

    // Set output directory (works for both local and Google Drive)
    const char *outputDir = "./";

    // Check if running in Google Colab with mounted Drive
    FILE *testDrive = fopen("/content/drive/MyDrive/test", "w");
    if (testDrive != NULL) {
        fclose(testDrive);
        remove("/content/drive/MyDrive/test");
        outputDir = "/content/drive/MyDrive/julia_set/";
        printf("Google Drive detected! Saving to: %s\n", outputDir);

        // Create output directory
        system("mkdir -p /content/drive/MyDrive/julia_set");
    } else {
        printf("Saving to local directory: %s\n", outputDir);
    }
    printf("\n");

    size_t imageSize = WIDTH * HEIGHT * sizeof(Color);

    // Allocate host memory
    Color *h_image_cpu = (Color*)malloc(imageSize);
    Color *h_image_gpu = (Color*)malloc(imageSize);

    printf("Rendering animated Julia set sequence...\n");
    printf("Parameters:\n");
    if (renderCPU) {
        printf("  CPU frames: %d to %d (%d total frames)\n", cpuStartFrame, cpuEndFrame, cpuNumFrames);
    } else {
        printf("  CPU rendering: DISABLED\n");
    }
    if (renderGPU) {
        printf("  GPU frames: %d to %d (%d total frames)\n", gpuStartFrame, gpuEndFrame, gpuNumFrames);
    } else {
        printf("  GPU rendering: DISABLED\n");
    }
    printf("  Cycle length: %d frames\n", cycleLength);
    printf("  Zoom center: (%.3f, %.3f)\n", centerX, centerY);
    printf("  Zoom range: %.1fx to %.1fx\n", zoomStart, zoomEnd);
    printf("  Animation stops at: %.0f%% of zoom\n\n", animationStopPercent * 100);

    // ============================================
    // CPU RENDERING
    // ============================================
    clock_t totalCPUTime = 0;
    double avgCPUTime = 0;

    if (renderCPU) {
        printf("CPU Rendering:\n");
        printf("------------------------------------------\n");

        // Open CSV file for CPU values
        char csvFilename[200];
        sprintf(csvFilename, "%scpu_values.csv", outputDir);
        printf("Creating CSV: %s\n", csvFilename);
        FILE *csvFile = fopen(csvFilename, "w");
        if (csvFile == NULL) {
            printf("ERROR: Could not create CPU CSV file!\n");
        } else {
            fprintf(csvFile, "frame,centerX,centerY,zoom\n");
        }

        for (int i = 0; i < cpuNumFrames; i++) {
            int frameNum = cpuStartFrame + i;

            // Animate the c parameter for morphing effect
            double angle = 2 * PI * frameNum / (double)cycleLength;

            // Stop animation at specified percent through the zoom for stability
            double zoomFactor = (frameNum - 1.0) / cycleLength;
            double dampening;
            if (zoomFactor < animationStopPercent) {
                // Normal animation
                dampening = 1.0;
            } else {
                // Freeze animation at deep zoom
                dampening = 0.0;
                // Lock angle to what it was at stop point
                angle = 2 * PI * (animationStopPercent * cycleLength) / (double)cycleLength;
            }

            double cReal = -0.7 + 0.2 * cos(angle) * dampening;
            double cImag = 0.27015 + 0.1 * sin(angle) * dampening;

            // Calculate zoom
            double t = (double)(frameNum - 1) / (double)cycleLength;
            double zoom = zoomStart * pow(zoomEnd / zoomStart, t);

            // Scale iterations with zoom for better detail at high magnification
            int iterations = (int)(MAX_ITER * (1.0 + log10(zoom)));
            if (iterations > MAX_ITER * 4) iterations = MAX_ITER * 4;  // Cap at 4x

            // Write to CSV with high precision
            if (csvFile != NULL) {
                fprintf(csvFile, "%d,%.15f,%.15f,%.6f\n", frameNum, centerX, centerY, zoom);
            }

            clock_t start = clock();
            juliaSetCPU(h_image_cpu, WIDTH, HEIGHT, centerX, centerY, zoom, cReal, cImag, iterations);
            clock_t end = clock();

            double frameTime = ((double)(end - start)) / CLOCKS_PER_SEC;
            totalCPUTime += (end - start);

            printf("Frame %d (zoom: %.1fx): %.3f seconds\n", frameNum, zoom, frameTime);

            // Save frame
            char filename[200];
            sprintf(filename, "%sjulia_cpu_%04d.ppm", outputDir, frameNum);
            savePPM(filename, h_image_cpu, WIDTH, HEIGHT);
        }

        avgCPUTime = ((double)totalCPUTime / CLOCKS_PER_SEC) / cpuNumFrames;
        float cpuFPS = 1.0 / avgCPUTime;
        printf("\nAverage CPU time: %.3f seconds\n", avgCPUTime);
        printf("CPU FPS: %.2f\n\n", cpuFPS);

        // Write FPS and close CSV
        if (csvFile != NULL) {
            fprintf(csvFile, "fps,%.2f,0,0\n", cpuFPS);
            fclose(csvFile);
            printf("Saved: %scpu_values.csv\n\n", outputDir);
        }
    } else {
        printf("CPU Rendering: SKIPPED\n\n");
    }

    // ============================================
    // GPU RENDERING
    // ============================================
    float avgGPUTime = 0;

    if (renderGPU) {
        printf("GPU Rendering:\n");
        printf("------------------------------------------\n");

        // Allocate device memory
        Color *d_image;
        cudaMalloc(&d_image, imageSize);

        // Setup kernel launch parameters
        dim3 threadsPerBlock(16, 16);
        dim3 blocksPerGrid((WIDTH + 15) / 16, (HEIGHT + 15) / 16);

        printf("Grid: %dx%d blocks, Block: %dx%d threads\n",
               blocksPerGrid.x, blocksPerGrid.y, threadsPerBlock.x, threadsPerBlock.y);
        printf("Total GPU threads: %d\n\n",
               blocksPerGrid.x * blocksPerGrid.y * threadsPerBlock.x * threadsPerBlock.y);

        // Create CUDA events for timing
        cudaEvent_t start, stop;
        cudaEventCreate(&start);
        cudaEventCreate(&stop);

        // Open CSV file for GPU values
        char csvFilename[200];
        sprintf(csvFilename, "%sgpu_values.csv", outputDir);
        printf("Creating CSV: %s\n", csvFilename);
        FILE *csvFile = fopen(csvFilename, "w");
        if (csvFile == NULL) {
            printf("ERROR: Could not create GPU CSV file!\n");
        } else {
            fprintf(csvFile, "frame,centerX,centerY,zoom\n");
        }

        float totalGPUTime = 0.0f;

        for (int i = 0; i < gpuNumFrames; i++) {
            int frameNum = gpuStartFrame + i;

            // Animate the c parameter for morphing effect
            double angle = 2 * PI * frameNum / (double)cycleLength;

            // Stop animation at specified percent through the zoom for stability
            double zoomFactor = (frameNum - 1.0) / cycleLength;
            double dampening;
            if (zoomFactor < animationStopPercent) {
                // Normal animation
                dampening = 1.0;
            } else {
                // Freeze animation at deep zoom
                dampening = 0.0;
                // Lock angle to what it was at stop point
                angle = 2 * PI * (animationStopPercent * cycleLength) / (double)cycleLength;
            }

            double cReal = -0.7 + 0.2 * cos(angle) * dampening;
            double cImag = 0.27015 + 0.1 * sin(angle) * dampening;

            // Calculate zoom
            double t = (double)(frameNum - 1) / (double)cycleLength;
            double zoom = zoomStart * pow(zoomEnd / zoomStart, t);

            // Scale iterations with zoom for better detail at high magnification
            int iterations = (int)(MAX_ITER * (1.0 + log10(zoom)));
            if (iterations > MAX_ITER * 4) iterations = MAX_ITER * 4;  // Cap at 4x

            // Write to CSV with high precision
            if (csvFile != NULL) {
                fprintf(csvFile, "%d,%.15f,%.15f,%.6f\n", frameNum, centerX, centerY, zoom);
            }

            cudaEventRecord(start);

            juliaSetGPU<<<blocksPerGrid, threadsPerBlock>>>(
                d_image, WIDTH, HEIGHT, centerX, centerY, zoom, cReal, cImag, iterations);

            cudaEventRecord(stop);
            cudaEventSynchronize(stop);

            // Check for errors
            cudaError_t err = cudaGetLastError();
            if (err != cudaSuccess) {
                printf("Kernel error: %s\n", cudaGetErrorString(err));
                break;
            }

            float frameTime;
            cudaEventElapsedTime(&frameTime, start, stop);
            totalGPUTime += frameTime;

            printf("Frame %d (zoom: %.1fx): %.3f seconds\n", frameNum, zoom, frameTime / 1000.0);

            // Copy result and save frame
            cudaMemcpy(h_image_gpu, d_image, imageSize, cudaMemcpyDeviceToHost);
            char filename[200];
            sprintf(filename, "%sjulia_gpu_%04d.ppm", outputDir, frameNum);
            savePPM(filename, h_image_gpu, WIDTH, HEIGHT);
        }

        avgGPUTime = (totalGPUTime / 1000.0) / gpuNumFrames;
        float gpuFPS = 1.0 / avgGPUTime;
        printf("\nAverage GPU time: %.3f seconds\n", avgGPUTime);
        printf("GPU FPS: %.2f\n\n", gpuFPS);

        // Write FPS and close CSV
        if (csvFile != NULL) {
            fprintf(csvFile, "fps,%.2f,0,0\n", gpuFPS);
            fclose(csvFile);
            printf("Saved: %sgpu_values.csv\n\n", outputDir);
        }

        // Cleanup GPU resources
        cudaFree(d_image);
        cudaEventDestroy(start);
        cudaEventDestroy(stop);
    } else {
        printf("GPU Rendering: SKIPPED\n\n");
    }

    // ============================================
    // PERFORMANCE COMPARISON
    // ============================================
    printf("===========================================\n");
    printf("PERFORMANCE SUMMARY\n");
    printf("===========================================\n");

    if (renderCPU && renderGPU) {
        printf("Average CPU time: %.3f seconds (%.2f FPS)\n", avgCPUTime, 1.0 / avgCPUTime);
        printf("Average GPU time: %.3f seconds (%.2f FPS)\n", avgGPUTime, 1.0 / avgGPUTime);
        printf("Speedup:          %.2fx faster on GPU\n", avgCPUTime / avgGPUTime);
    } else if (renderCPU) {
        printf("Average CPU time: %.3f seconds (%.2f FPS)\n", avgCPUTime, 1.0 / avgCPUTime);
        printf("GPU rendering was skipped\n");
    } else if (renderGPU) {
        printf("Average GPU time: %.3f seconds (%.2f FPS)\n", avgGPUTime, 1.0 / avgGPUTime);
        printf("CPU rendering was skipped\n");
    }
    printf("===========================================\n\n");

    if (renderGPU && avgGPUTime < 0.033) {
        printf("✓ GPU can render at 30+ FPS!\n");
        if (avgGPUTime < 0.016) {
            printf("✓ GPU can render at 60+ FPS!\n");
        }
    }

    printf("\nOutput files saved to: %s\n", outputDir);
    if (renderCPU) {
        printf("CPU: %d frames (%d to %d)\n", cpuNumFrames, cpuStartFrame, cpuEndFrame);
    }
    if (renderGPU) {
        printf("GPU: %d frames (%d to %d)\n", gpuNumFrames, gpuStartFrame, gpuEndFrame);
    }

    // Cleanup
    free(h_image_cpu);
    free(h_image_gpu);

    return 0;
}

Overwriting julia_set.cu


In [39]:
from google.colab import drive
drive.mount('/content/drive')

!nvcc -arch=sm_75 -o julia_set julia_set.cu
!./julia_set --start 1 --cpu-end 30 --gpu-end 300 --cycle-length 300 --zoom 300 --animation-stop 0.4 --centerX  0.11230583918508258 --centerY -0.17749724081559552

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
JULIA SET: CPU vs GPU Performance
Resolution: 1920x1080 (2 megapixels)

Google Drive detected! Saving to: /content/drive/MyDrive/julia_set/

Rendering animated Julia set sequence...
Parameters:
  CPU frames: 1 to 30 (30 total frames)
  GPU frames: 1 to 300 (300 total frames)
  Cycle length: 300 frames
  Zoom center: (0.112, -0.177)
  Zoom range: 1.0x to 300.0x
  Animation stops at: 40% of zoom

CPU Rendering:
------------------------------------------
Creating CSV: /content/drive/MyDrive/julia_set/cpu_values.csv
Frame 1 (zoom: 1.0x): 1.344 seconds
Saved: /content/drive/MyDrive/julia_set/julia_cpu_0001.ppm
Frame 2 (zoom: 1.0x): 1.377 seconds
Saved: /content/drive/MyDrive/julia_set/julia_cpu_0002.ppm
Frame 3 (zoom: 1.0x): 1.299 seconds
Saved: /content/drive/MyDrive/julia_set/julia_cpu_0003.ppm
Frame 4 (zoom: 1.1x): 1.348 seconds
Saved: /content/drive/MyDrive/ju

In [15]:
!grep -c "cpu_values.csv" julia_set.cu
!grep -c "gpu_values.csv" julia_set.cu

2
2


In [40]:

from google.colab import files

from PIL import Image, ImageDraw, ImageFont
import csv
import os

input_dir = '/content/drive/MyDrive/julia_set/'

print("WARNING: This will overwrite your original frames!")
print("Checking for CSV files...\n")

# Define both CPU and GPU configurations
configs = [
    {'csv': 'cpu_values.csv', 'prefix': 'julia_cpu_', 'name': 'CPU', 'output': 'cpu_julia_labeled.mp4'},
    {'csv': 'gpu_values.csv', 'prefix': 'julia_gpu_', 'name': 'GPU', 'output': 'gpu_julia_labeled.mp4'}
]

# Process each configuration if CSV exists
for config in configs:
    csv_file = f'{input_dir}{config["csv"]}'

    if not os.path.exists(csv_file):
        print(f"Skipping {config['name']}: {config['csv']} not found")
        continue

    print(f"\n{'='*50}")
    print(f"Processing {config['name']} frames from {config['csv']}")
    print(f"{'='*50}")

    # Read CSV file
    frame_data = {}
    fps = 30.0  # Default fallback

    with open(csv_file, 'r') as f:
        reader = csv.DictReader(f)
        for row in reader:
            if row['frame'] == 'fps':
                # Last line contains FPS
                fps = float(row['centerX'])
                print(f"Found {config['name']} FPS: {fps:.2f}")
            else:
                frame_num = int(row['frame'])
                frame_data[frame_num] = {
                    'centerX': float(row['centerX']),
                    'centerY': float(row['centerY']),
                    'zoom': float(row['zoom'])
                }

    num_frames = len(frame_data)
    print(f"Found {num_frames} frames in CSV")

    # Process each frame
    for frame_num, data in frame_data.items():
        img_path = f'{input_dir}{config["prefix"]}{frame_num:04d}.ppm'

        if not os.path.exists(img_path):
            print(f"Skipping frame {frame_num} - file not found")
            continue

        img = Image.open(img_path)
        draw = ImageDraw.Draw(img)

        # Try to load a nice font, fallback to default
        try:
            font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf", 24)
        except:
            font = ImageFont.load_default()

        # Create text using values from CSV
        text = f"Frame {frame_num} | Center ({data['centerX']:.15f}, {data['centerY']:.15f}) | Zoom {data['zoom']:.3f}x"

        # Get text size for background box
        bbox = draw.textbbox((0, 0), text, font=font)
        text_width = bbox[2] - bbox[0]
        text_height = bbox[3] - bbox[1]

        # Draw semi-transparent background box
        box_padding = 10
        box_coords = [
            10,
            img.height - text_height - box_padding - 10,
            text_width + box_padding + 20,
            img.height - 10
        ]
        draw.rectangle(box_coords, fill=(0, 0, 0, 180))

        # Draw text
        draw.text((15, img.height - text_height - 15), text, fill=(255, 255, 255), font=font)

        # Save back to original file
        img.save(img_path)

        if frame_num % 10 == 0:
            print(f"Processed frame {frame_num}")

    print(f"\n{config['name']} frames complete!")

    # Create video for this set using calculated FPS and actual frame count
    start_frame = min(frame_data.keys())
    end_frame = max(frame_data.keys())

    output_path = f'{input_dir}{config["output"]}'

    print(f"Creating {config['name']} video...")
    print(f"  Start frame: {start_frame}")
    print(f"  End frame: {end_frame}")
    print(f"  Total frames: {num_frames}")
    print(f"  FPS: {fps:.2f}")

    !ffmpeg -y -framerate {fps} -start_number {start_frame} \
            -i {input_dir}{config["prefix"]}%04d.ppm \
            -frames:v {num_frames} -c:v libx264 -pix_fmt yuv420p \
            {output_path}

    print(f"{config['name']} video created: {output_path}\n")

print("\n" + "="*50)
print("All processing complete!")
print("="*50)

files.download('/content/drive/MyDrive/julia_set/cpu_julia_labeled.mp4')
files.download('/content/drive/MyDrive/julia_set/gpu_julia_labeled.mp4')


Checking for CSV files...


Processing CPU frames from cpu_values.csv
Found CPU FPS: 0.45
Found 30 frames in CSV
Processed frame 10
Processed frame 20
Processed frame 30

CPU frames complete!
Creating CPU video...
  Start frame: 1
  End frame: 30
  Total frames: 30
  FPS: 0.45
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --ena

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>